### Outline

- *Load the CIFAR-10 Dataset*
- *Dataset Preprocessing*
- *Dataset and Training Configuration Parameters*
- *CNN Model Implementation in Keras*
- *Adding Dropout to the Model*
- *Saving and Loading Models*
- *Model Evaluation*


In [ ]:
# Import basic libraries
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Set random seeds for reproducibility
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
os.environ['PYTHONHASHSEED'] = str(seed_value)

In [ ]:
# Set random seeds for reproducibility in TensorFlow
import tensorflow as tf
tf.random.set_seed(seed_value)

In [ ]:
## Load the CIFAR-10 dataset
import tensorflow as tf
from tensorflow.keras.datasets import cifar10

dataset = cifar10.load_data()

In [ ]:
dataset

In [ ]:
dataset[0][0]

In [ ]:
# 50000 training images, each of size 32x32 with 3 color channels
# 32 The hight and width of each image in pixels
# 3 The number of color channels (RGB)

dataset[0][0].shape  # (50000, 32, 32, 3)

In [ ]:
dataset[0][0][0]

In [ ]:
# 32: The hight of the image in pixels
# 32: The width of the image in pixels
# 3: The number of color channels (RGB)

dataset[0][0][0].shape

In [ ]:
# Visualise the number of channels in the first image
data = dataset[0][0][0]  # First image in the training set

print(f"Data shape: {data.shape}")

# Scatter Plot (for sparse data or voxels)
fig = plt.figure(figsize=(30, 10))

ax1 = fig.add_subplot(131, projection='3d')
x, y, z = np.where(data > 0.5)  # Show points above threshold
ax1.scatter(x, y, z, c=data[x, y, z], cmap='viridis', marker='o')
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('3D Scatter Plot')
plt.show()

In [ ]:
# Get the training and testing data from the dataset
(X_train, y_train), (X_test, y_test) = dataset

In [ ]:
# The CIFAR-10 dataset consists of 50,000 training images and 10,000 test images
# with 10 classes.
print(f"X_train Shape: {X_train.shape}")
print(f"y_train Shape: {y_train.shape}")
print(f"X_test Shape: {X_test.shape}")
print(f"y_test Shape: {y_test.shape}")

In [ ]:
# Visualise sample images from the dataset

plt.figure(figsize=(18, 9))

num_rows = 4
num_cols = 8

# plot each of the images in th ebatch and the assoficated label
for i in range(num_rows*num_cols):
    plt.subplot(num_rows, num_cols, i+1)
    plt.imshow(X_train[i].astype("uint8"))
    plt.title(f"Label: {y_train[i][0]}")
    plt.axis("off")

## Data Preprocessing

In [ ]:
# Normalize pixel values to be between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0   

In [ ]:
# Change the labels from integer to categorical data.
print("Original (integer) label for the first training sample:", y_train[0])

In [ ]:
# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
y_train

In [ ]:
print('After conversion to categorical one-hot encoded labels: ', y_train[0])


### NOTE: 

- **The @dataclass(frozen=True) decorator, is a clean and standard way to define a Configuration Class for a Machine Learning or Deep Learning training script.**

- **The primary use of this structure is to centralize, organize, and manage the hyperparameters used to train a machine learning model.**

In [ ]:
## Dataset and Training Configuration Parameters`
from dataclasses import dataclass

@dataclass(frozen=True)
class DatasetConfig:
    NUM_CLASSES: int = 10
    IMAGE_HEIGHT: int = 32
    IMAGE_WIDTH: int = 32
    IMAGE_CHANNELS: int = 3

@dataclass(frozen=True)
class TrainingConfig:
    BATCH_SIZE: int = 64
    EPOCHS: int = 25
    LEARNING_RATE: float = 0.001
    VALIDATION_SPLIT: float = 0.3  

### CNN Model Implementation in Keras

- **Build/Define a network model using predefined layers in Keras.**
- **Compile the model with model.compile()**
- **Train the model with model.fit()**

In [ ]:
# Building the model using the sequential API

from tensorflow.keras import layers, models

model = models.Sequential()



# First Conv Layer (32 filters, 3x3 kernel, ReLU activation, input shape)
input_shape=(32, 32, 3)
model.add(layers.InputLayer(input_shape=input_shape))
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Second Conv Layer (64 filters, 3x3 kernel, ReLU activation)
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Third Conv Layer (64 filters, 3x3 kernel, ReLU activation)
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and add Dense layers
model.add(layers.Flatten())

# First Fully Connected Layer (512 units, ReLU activation)
model.add(layers.Dense(units=512, activation='relu'))

# Output Layer (10 units for 10 classes, Softmax activation)
model.add(layers.Dense(units=10, activation='softmax'))

In [ ]:
model.summary()

## Compile the Model

Compile the model by specifying the optimiser type and loss function and any additional metrics we would like to record during training.

- Optimiser: **RMSProp**
- loss: **Categorical Cross-entropy**
- metrics: **Accuracy**

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=TrainingConfig.LEARNING_RATE),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    X_train, y_train,
    batch_size=TrainingConfig.BATCH_SIZE,
    epochs=TrainingConfig.EPOCHS,
    validation_split=TrainingConfig.VALIDATION_SPLIT,
    shuffle=True
)

## Model Evaluation

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Building the model using the sequential API

from tensorflow.keras import layers, models

model2 = models.Sequential()



# First Conv Layer (32 filters, 3x3 kernel, ReLU activation, input shape)
input_shape=(32, 32, 3)
model2.add(layers.InputLayer(input_shape=input_shape))
model2.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
model2.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Dropout(0.25))

# Second Conv Layer (64 filters, 3x3 kernel, ReLU activation)
model2.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model2.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Dropout(0.25))


# Third Conv Layer (64 filters, 3x3 kernel, ReLU activation)
model2.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model2.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Dropout(0.25))

# Flatten the output and add Dense layers
model2.add(layers.Flatten())

# First Fully Connected Layer (512 units, ReLU activation)
model2.add(layers.Dense(units=512, activation='relu'))
model2.add(layers.Dropout(0.5))


# Output Layer (10 units for 10 classes, Softmax activation)
model2.add(layers.Dense(units=10, activation='softmax'))

In [ ]:
model2.summary()

## Compile the Model Again

In [ ]:
model2.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
history2 = model2.fit(
    X_train, y_train,
    batch_size=TrainingConfig.BATCH_SIZE,
    epochs=TrainingConfig.EPOCHS,
    validation_split=TrainingConfig.VALIDATION_SPLIT,
    shuffle=True
)